In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as pt
%matplotlib inline

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [11]:
n_cell_x = 20
n_cell_y = 40 

size_x = 10. / n_cell_x
size_y = 10. / n_cell_y
eps = 0.00001  
xs = np.where(df.x.values < eps, 0, df.x.values - eps)
ys = np.where(df.y.values < eps, 0, df.y.values - eps)
pos_x = (xs / size_x).astype(np.int)
pos_y = (ys / size_y).astype(np.int)

df['grid_cell'] = pos_y * n_cell_x + pos_x

In [15]:
def process_one_cell(df_train, df_test=None, grid_id, th):
    """   
    Classification inside one grid cell.
    """   
    #Working on df_train
    df_cell_train = df_train.loc[df_train.grid_cell == grid_id]
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]
    
    ##For production:
    if (df_test != None) and len(df_test) > 1:
        ##Preparing train data:
        le = LabelEncoder()
        y = le.fit_transform(df_cell_train.place_id.values)
        X = df_cell_train.drop(['place_id', 'grid_cell'], axis=1).values.astype(int)
        
        ##Preparing test data:
        df_cell_test = df_test.loc[df_test.grid_cell == grid_id]
        row_ids = df_cell_test.index
        X_test = df_cell_test.drop(['grid_cell'], axis = 1).values.astype(int)
    else:
        ##For test:
        train, test = train_test_split(df_cell_train, test_size=0.2)
        le = LabelEncoder()
        y = le.fit_transform(train.place_id.values)
        X = train.drop(['place_id', 'grid_cell'], axis=1).values.astype(int)
        
        row_ids = test.index
        y_test = le.transform(test.place_id.values)
        X_test = test.drop(['place_id', 'grid_cell'], axis = 1).values.astype(int)
        
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=25, weights='distance', 
                               metric='manhattan')
    clf.fit(X, y)
    y_pred = clf.predict_proba(X_test)
    pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:3])    
    return pred_labels, row_ids